# SymPy for solving Euler-Bernoulli

SymPy can be used to solve the Euler-Bernoulli's beam theory. It will take over the cumbersome handcalculations. However the engineering part; the thinking, the modelling and the correct input of the specifications still remains a human matter. 

A simple beam is taken as an example. The beam is loaded with a uniform load `q` on a part of the beam and a force `F`. The beam is a prismatic beam with a bending stiffness `EI`.

```{figure} ./sympy_mech_data/beam.svg
:align: center
```

We can solve this beam by using the [Euler-Bernoulli's differential equation](beam_force_ode).

First, the SymPy library is imported and a function is called which will make sure the output looks nice:

In [2]:
import sympy as sym
sym.init_printing()

Now, we'll define our symbols. These includes all symbols (non-numerical values), so also including intergration constants. We do so with the `sym.symbols` function:

In [5]:
x, q, F, EI = sym.symbols('x, q, F, EI')
C1, C2, C3, C4, C5, C6, C7, C8, C9, C10, C11, C12 = sym.symbols('C1 C2 C3 C4 C5 C6 C7 C8 C9 C10 C11 C12')

The functions $q\left(x\right)$ for each of the segments can now be defined:

In [6]:
q_AC = q
q_CD = 0
q_DB = 0

We can integrate using the `sym.integrate()` function:

In [10]:
V_AC = sym.integrate(-q_AC,x)+C1
M_AC = sym.integrate(V_AC,x)+C2
kappa_AC = M_AC / EI
phi_AC = sym.integrate(kappa_AC,x)+C3
w_AC = sym.integrate(-phi_AC,x)+C4

V_CD = sym.integrate(-q_CD,x)+C5
M_CD = sym.integrate(V_CD,x)+C6
kappa_CD = M_CD / EI
phi_CD = sym.integrate(kappa_CD,x)+C7
w_CD = sym.integrate(-phi_CD,x)+C8

V_DB = sym.integrate(-q_DB,x)+C9
M_DB = sym.integrate(V_DB,x)+C10
kappa_DB = M_DB / EI
phi_DB = sym.integrate(kappa_DB,x)+C11
w_DB = sym.integrate(-phi_DB,x)+C12

You can display the equations with the `display` function:

In [9]:
display(w_AC)

-C1*x**3/(6*EI) - C2*x**2/(2*EI) - C3*x + C4 + q*x**4/(24*EI)

The boundary conditions can now be specified using the `sym.Eq()` function which takes as an input the left- and righthandside of an equation. Furthermore the `.subs()` function fill in a value in your expression:

In [12]:
Eq1 = sym.Eq(w_AC.subs(x, 0), 0) 
Eq2 = sym.Eq(M_AC.subs(x, 0), 0)
Eq3 = sym.Eq(w_AC.subs(x, 4), w_CD.subs(x, 4))
Eq4 = sym.Eq(phi_AC.subs(x, 4), phi_CD.subs(x, 4))
Eq5 = sym.Eq(M_AC.subs(x, 4), M_CD.subs(x, 4))
Eq6 = sym.Eq(V_AC.subs(x, 4), V_CD.subs(x, 4))
Eq7 = sym.Eq(w_CD.subs(x, 8), w_DB.subs(x, 8))
Eq8 = sym.Eq(phi_CD.subs(x, 8), phi_DB.subs(x, 8))
Eq9 = sym.Eq(V_CD.subs(x, 8), V_DB.subs(x, 8)+F)
Eq10 = sym.Eq(M_CD.subs(x, 8), M_DB.subs(x, 8))
Eq11 = sym.Eq(w_DB.subs(x, 12), 0)
Eq12 = sym.Eq(M_DB.subs(x, 12), 0)

display(Eq7)

Eq(-256*C5/(3*EI) - 32*C6/EI - 8*C7 + C8, -32*C10/EI - 8*C11 + C12 - 256*C9/(3*EI))

Now we use the function `sym.solve` to solve our system of equations for our integration constants:

In [13]:
sol = sym.solve((Eq1,Eq2,Eq3,Eq4,Eq5,Eq6,Eq7,Eq8,Eq9,Eq10,Eq11,Eq12), (C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12))
display(sol)

{C1: F/3 + 10*q/3,
 C10: 8*F + 8*q,
 C11: (-352*F - 296*q)/(9*EI),
 C12: (-256*F - 32*q)/(3*EI),
 C2: 0,
 C3: (-64*F - 200*q)/(9*EI),
 C4: 0,
 C5: F/3 - 2*q/3,
 C6: 8*q,
 C7: (-64*F - 296*q)/(9*EI),
 C8: -32*q/(3*EI),
 C9: -2*F/3 - 2*q/3}

We can use `.subs()` again to substitute this solution in our original function:

In [14]:
display(w_AC.subs(sol))

q*x**4/(24*EI) - x**3*(F/3 + 10*q/3)/(6*EI) - x*(-64*F - 200*q)/(9*EI)

Instead of symbolic values, you could have worked with numerical values too from the beginning. Nevertheless, we can substitute values with out final expression too, including a value for x. Use `.evalf()` to show the decimal form of your answer:

In [20]:
display(w_CD.subs(sol).subs(EI,20000).subs(q,10).subs(F,10).subs(x,6))
display(w_CD.subs(sol).subs(EI,20000).subs(q,10).subs(F,10).subs(x,6).evalf())

73/1500

0.0486666666666667

## Exercises
- Exercises in chapter 11.4 of the book Engineering Mechanics Volume 1 {cite:p}`Hartsuijker2006`, answers are available on [this website](https://icozct.tudelft.nl/TUD_CT/bookanswers/vol1/Chapter11/): 11.1 - 11.11